In [5]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
conn = psycopg2.connect(database=os.environ["DATABASE"],host="localhost",user=os.environ["POSTGRES_USER"],password=os.environ["POSTGRES_PASSWORD"],port="5432")

### Number of Trips Everyday


In [ ]:
df= pd.read_sql("""

select tpep_pickup_datetime::date as pickup_date,count(*) from
fact_trips
group by pickup_date
order by 1
                """,conn)
df

C:\Users\Goku\AppData\Local\Temp\ipykernel_10668\3545048497.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql("""


,pickup_date,count
0,2024-12-31,21
1,2025-01-01,74364
2,2025-01-02,81405
3,2025-01-03,87655
4,2025-01-04,93311
5,2025-01-05,75853
6,2025-01-06,76041
7,2025-01-07,92165
8,2025-01-08,99707
9,2025-01-09,103898


### Total Sales by Payment Type

In [12]:
df= pd.read_sql("""
select payment_id,payment_method,floor(sum(total_amount)) as total_amount
from
fact_trips 
left join dim_payment dp on dp.payment_id = fact_trips.payment_type
where total_amount>0
group by 1,2
                """,conn)
df

C:\Users\Goku\AppData\Local\Temp\ipykernel_10668\1189438227.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql("""


,payment_id,payment_method,total_amount
0,1,Credit card,68637075.0
1,2,Cash,8884569.0
2,3,No charge,346446.0
3,4,Dispute,1923302.0


### Finding the longest journey on each day


In [13]:
df= pd.read_sql("""
with MaxJourneys as (
select
	tpep_pickup_datetime::date as pickup_date,
	pu.zone as Pickup_Zone,
	df.zone as Dropoff_Zone,
	ft.journeyminutes,
	row_number() over (partition by tpep_pickup_datetime::date order by ft.journeyminutes desc) as ranked
from
	fact_trips ft
left join
	dim_zone pu on pu.locationid = ft."PULocationID"
left join 
	dim_zone df on df.locationid= ft."DOLocationID"
group by 1,2,3,4
)
select * from MaxJourneys 
where ranked=1
                """,conn)
df

C:\Users\Goku\AppData\Local\Temp\ipykernel_10668\1601426019.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql("""


,pickup_date,pickup_zone,dropoff_zone,journeyminutes,ranked
0,2024-12-31,JFK Airport,Lincoln Square East,96,1
1,2025-01-01,Midtown South,Whitestone,2485,1
2,2025-01-02,Queensbridge/Ravenswood,Park Slope,2876,1
3,2025-01-03,Midtown North,Upper East Side South,1438,1
4,2025-01-04,Midtown East,Roosevelt Island,1438,1
5,2025-01-05,Hudson Sq,SoHo,1438,1
6,2025-01-06,Springfield Gardens North,Queensbridge/Ravenswood,5022,1
7,2025-01-07,Outside of NYC,Lenox Hill West,2764,1
8,2025-01-08,Midtown Center,Penn Station/Madison Sq West,1438,1
9,2025-01-09,Midtown Center,West Chelsea/Hudson Yards,1439,1
